# Fetch and Process GitHub Security Advisories

This notebook is used to create the RAG knowledge base based on github advisories. 

it  demonstrates how to fetch and process security advisories from GitHub repositories using the GitHub API. Security advisories provide important information about vulnerabilities and their mitigations, which is crucial for maintaining the security of software projects.

## Setup

Before running the notebook, ensure you have the following:

1. **GitHub API Token**: You need a GitHub API token to access the GitHub API. Store this token in a `.env` file with the key `GITHUB_API_TOKEN`.
2. **Required Libraries**: Install the necessary Python libraries, including `requests`, `pandas`, `dotenv`, and any other dependencies.

## Steps

1. **Load Environment Variables**: Load the GitHub API token from the `.env` file.
2. **Fetch Security Advisories**: Define functions to interact with the GitHub API and fetch security advisories from specified repositories.
3. **Process Advisories**: Parse and process the fetched advisories to extract relevant information such as advisory ID, description, severity, affected versions, etc.
4. **Store Advisories**: Embed the Save the processed advisories in a structured format (e.g., CSV file) for further analysis.


## Usage

Follow the steps in the notebook to fetch, process, and analyze security advisories from GitHub repositories. 

Let's get started!

In [1]:
import requests
import os
import json
from dotenv import load_dotenv

import base64
import psycopg2
from langchain_core.documents import Document
from langchain_postgres import PGVector
from langchain_postgres.vectorstores import PGVector
from langchain_openai import OpenAIEmbeddings
from tqdm import tqdm
import asyncio
# Load the environment variables from the .env file
load_dotenv()

True

In [2]:
# Set up the API token and endpoint
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
URL = "https://api.github.com/advisories"

# Define the GraphQL query
# Query Parameters (e.g., fetch all critical advisories for Python)
params = {
    "type": "reviewed",
    "ecosystem ": "pip",
    "per_page": 100,
}

In [3]:
# Headers (for authentication)
headers = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github.v3+json"
}

In [4]:
def get_advisories_info(advisories):
    advisory_documents = []

    for advisory in tqdm(advisories):
        metadata = {
            "cve_id": advisory.get("cve_id", "N/A"),
            "severity": advisory.get("severity", "N/A"),
            "source_code_location": advisory.get("source_code_location", "N/A"),
        }

        # Extract vulnerable packages
        vulnerable_packages = advisory.get("vulnerabilities", [])
        package_list = []
        
        for vuln in vulnerable_packages:
            package_name = vuln["package"]["name"]
            vulnerable_version_range = vuln["vulnerable_version_range"]
            vulnerable_functions = vuln.get("vulnerable_functions", []) or ["NA"]
            
            package_list.append({
                "Package Name": package_name,
                "Vulnerable Version Range": vulnerable_version_range,
                "Vulnerable Functions": ", ".join(vulnerable_functions)
            })

        # Store extracted data
        metadata["packages"] = package_list

        content = f"Advisory summary: {advisory.get('summary', 'N/A')}\n Advisory description: {advisory.get('description', 'N/A')}"
        
        advisory_documents.append(Document(page_content=content, metadata=metadata))

    return advisory_documents


In [5]:
# Initialize OpenAI Embeddings
embeddings = OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"))
# PostgreSQL PGVector Config
DB_PARAMS = {
    "database": "malware_kb",
    "user": "malware_admin",
    "password": "admin_secure_password",
    "host": "localhost",
    "port": "5432"
}

PGVECTOR_CONNECTION_STRING = f"postgresql+psycopg://{DB_PARAMS['user']}:{DB_PARAMS['password']}@{DB_PARAMS['host']}:{DB_PARAMS['port']}/{DB_PARAMS['database']}?options=-csearch_path=malware"

def store_in_pgvector(docs):
    vector_store = PGVector(
        connection=PGVECTOR_CONNECTION_STRING,  # ✅ Use correct connection string
        embeddings=embeddings,
        collection_name="github_advisories",
        use_jsonb=True,
    )

    
    vector_store.add_documents(docs)  # ✅ Store asynchronously

    




In [9]:
# Pagination logic
advisories_num = 0
test=[]
page = 1
while URL:
    params = {
        "type": "reviewed",
        "ecosystem":"pip",
        "per_page": 100,
    }
    response = requests.get(URL, headers=headers, params=params)
        
    if response.status_code != 200:
        print(f"Error: {response.status_code} - {response.text}")
        break

    data = response.json()
    
    if not data:  # No more advisories left
        break
    
    advisories_num+=len(data)
    print(f"🔍 Fetching advisories from page {page}")
    # Extract advisory information
    advisory_documents = get_advisories_info(data)
    store_in_pgvector(advisory_documents)
    page+=1
    
    # Extract the "Link" header from the response
    link_header = response.headers.get("Link", "")

    # Find the next page URL
    next_url = None
    if link_header:
        links = link_header.split(", ")
        for link in links:
            if 'rel="next"' in link:
                next_url = link.split(";")[0].strip("<>")  # Extract URL
                break

    URL = next_url  # Update URL for next request or exit if no "next"


print(f"✅ Fetched {advisories_num} advisories")

🔍 Fetching advisories from page 1


100%|██████████| 100/100 [00:00<00:00, 15366.57it/s]


🔍 Fetching advisories from page 2


100%|██████████| 100/100 [00:00<?, ?it/s]


🔍 Fetching advisories from page 3


...

🔍 Fetching advisories from page 31


100%|██████████| 100/100 [00:00<?, ?it/s]


🔍 Fetching advisories from page 32


100%|██████████| 71/71 [00:00<00:00, 71005.15it/s]


✅ Fetched 3171 advisories


In [8]:
test[1]

{'ghsa_id': 'GHSA-g5vp-j278-8pjh',
 'cve_id': 'CVE-2024-49048',
 'url': 'https://api.github.com/advisories/GHSA-g5vp-j278-8pjh',
 'html_url': 'https://github.com/advisories/GHSA-g5vp-j278-8pjh',
 'summary': 'TorchGeo Remote Code Execution Vulnerability',
 'description': 'TorchGeo Remote Code Execution Vulnerability',
 'type': 'reviewed',
 'severity': 'high',
 'repository_advisory_url': None,
 'source_code_location': 'https://github.com/microsoft/torchgeo',
 'identifiers': [{'value': 'GHSA-g5vp-j278-8pjh', 'type': 'GHSA'},
  {'value': 'CVE-2024-49048', 'type': 'CVE'}],
 'references': ['https://nvd.nist.gov/vuln/detail/CVE-2024-49048',
  'https://msrc.microsoft.com/update-guide/vulnerability/CVE-2024-49048',
  'https://github.com/microsoft/torchgeo/pull/2323',
  'https://github.com/microsoft/torchgeo/releases/tag/v0.6.1',
  'https://github.com/pypa/advisory-database/tree/main/vulns/torchgeo/PYSEC-2024-204.yaml',
  'https://github.com/advisories/GHSA-g5vp-j278-8pjh'],
 'published_at': '20